# Deploy Lora fined tuned LLaVA on SageMaker
As the time of writing, LLaVA is not yet fully integrated into HuggingFace's transformers library, it has to be loaded it from disk. We will create a tar.gz file containing the model weights and the inference code.
Reference: https://medium.com/@liltom.eth/deploy-llava-1-5-on-amazon-sagemaker-168b2efd2489

1. Create a folder structure required by SageMaker and download the model weights and adapter weights from S3 (if you're using the weights from Hugging Face, you can use snapshot_download https://huggingface.co/docs/huggingface_hub/guides/download)

In [ ]:
!mkdir -p lora-model/code
!mkdir -p lora-model/adapter_model

!aws s3 cp s3://YOUR_BUCKET/FULL_TUNE_JOB_ID/checkpoints/ lora-model/ --recursive
!aws s3 cp s3://YOUR_BUCKET/LORA_JOB_ID/checkpoints/ lora-model/adapter_model --recursive


2. Create infernece code and requirements
Reference: https://github.com/haotian-liu/LLaVA/blob/main/llava/model/builder.py#L57

In [ ]:
%%writefile lora-model/code/inference.py
import os
import requests
from PIL import Image
from io import BytesIO
import torch
from transformers import AutoTokenizer

from llava.model import LlavaLlamaForCausalLM
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, KeywordsStoppingCriteria

from llava.conversation import conv_templates, SeparatorStyle
from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
)


def model_fn(model_dir):
    kwargs = {"device_map": "auto"}
    kwargs["torch_dtype"] = torch.float16
    adapter_path = os.path.join(model_dir, "adapter_model")
    
    print("loading lora config")
    from llava.model.language_model.llava_llama import LlavaConfig
    lora_cfg_pretrained = LlavaConfig.from_pretrained(adapter_path)
    print("loading model")
    model = LlavaLlamaForCausalLM.from_pretrained(
        model_dir, low_cpu_mem_usage=True, config=lora_cfg_pretrained, **kwargs
    )
    token_num, tokem_dim = model.lm_head.out_features, model.lm_head.in_features
    if model.lm_head.weight.shape[0] != token_num:
        model.lm_head.weight = torch.nn.Parameter(torch.empty(token_num, tokem_dim, device=model.device, dtype=model.dtype))
        model.model.embed_tokens.weight = torch.nn.Parameter(torch.empty(token_num, tokem_dim, device=model.device, dtype=model.dtype))

    if os.path.exists(os.path.join(adapter_path, 'non_lora_trainables.bin')):
        non_lora_trainables = torch.load(os.path.join(adapter_path, 'non_lora_trainables.bin'), map_location='cpu')

    non_lora_trainables = {(k[11:] if k.startswith('base_model.') else k): v for k, v in non_lora_trainables.items()}
    if any(k.startswith('model.model.') for k in non_lora_trainables):
        non_lora_trainables = {(k[6:] if k.startswith('model.') else k): v for k, v in non_lora_trainables.items()}
    model.load_state_dict(non_lora_trainables, strict=False)

    from peft import PeftModel
    print('Loading LoRA weights...')
    model = PeftModel.from_pretrained(model, adapter_path)
    print('Merging LoRA weights...')
    model = model.merge_and_unload()
    print('Model is loaded...')

    print("loading tokenizer")
    tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False)

    vision_tower = model.get_vision_tower()
    if not vision_tower.is_loaded:
        vision_tower.load_model()
    vision_tower.to(device="cuda", dtype=torch.float16)
    image_processor = vision_tower.image_processor
    return model, tokenizer, image_processor


def predict_fn(data, model_and_tokenizer):
    # unpack model and tokenizer
    model, tokenizer, image_processor = model_and_tokenizer

    # get prompt & parameters
    image_file = data.pop("image", data)
    raw_prompt = data.pop("question", data)

    max_new_tokens = data.pop("max_new_tokens", 1024)
    temperature = data.pop("temperature", 0.2)
    conv_mode = data.pop("conv_mode", "llava_v1")

    if conv_mode == "raw":
        # use raw_prompt as prompt
        prompt = raw_prompt
        stop_str = "###"
    else:
        conv = conv_templates[conv_mode].copy()
        roles = conv.roles
        inp = f"{roles[0]}: {raw_prompt}"
        inp = (
            DEFAULT_IM_START_TOKEN
            + DEFAULT_IMAGE_TOKEN
            + DEFAULT_IM_END_TOKEN
            + "\n"
            + inp
        )
        conv.append_message(conv.roles[0], inp)
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()
        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2

    if image_file.startswith("http") or image_file.startswith("https"):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")

    disable_torch_init()
    image_tensor = (
        image_processor.preprocess(image, return_tensors="pt")["pixel_values"]
        .half()
        .cuda()
    )

    keywords = [stop_str]
    input_ids = (
        tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
        .unsqueeze(0)
        .cuda()
    )
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor,
            do_sample=True,
            temperature=temperature,
            max_new_tokens=max_new_tokens,
            use_cache=True,
            stopping_criteria=[stopping_criteria],
        )
    outputs = tokenizer.decode(
        output_ids[0, input_ids.shape[1] :], skip_special_tokens=True
    ).strip()
    return outputs

In [ ]:
%%writefile lora-model/code/requirements.txt
llava @ git+https://github.com/haotian-liu/LLaVA@v1.1.1

3. Tar and gzip the model and inference code

In [ ]:
!tar -cvf llava-lora.tar.gz --use-compress-program=pigz -C lora-model .

4. Upload to S3

In [ ]:
!aws s3 cp llava-lora.tar.gz s3://YOUR_BUCKET/

5. Create SageMaker inference endpoint from the model

In [ ]:
import sagemaker
from sagemaker.huggingface.model import HuggingFaceModel

role = sagemaker.get_execution_role()
sess = sagemaker.Session()

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://YOUR_BUCKET/llava-lora.tar.gz",      # path to your model and script
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version="4.28.1",  # transformers version used
   pytorch_version="2.0.0",       # pytorch version used
   py_version='py310',            # python version used
   model_server_workers=1
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
    container_startup_health_check_timeout=600, # increase timeout for large models
    model_data_download_timeout=600, # increase timeout for large models
)

6. Wait until model is fully deployed and test it.
The first invocation might time out as it lazy loads the model during the first call.

In [ ]:
data = {
    "image" : 'https://raw.githubusercontent.com/haotian-liu/LLaVA/main/images/llava_logo.png', 
    "question" : "Describe the image and color details.",
    # "max_new_tokens" : 1024,
    # "temperature" : 0.2,
    # "stop_str" : "###",
}

# the first call might time out as it lazy loads the model during the first call
output = predictor.predict(data)
print(output)

7. If the model and endpoint are no longer needed, you can delete them

In [ ]:
predictor.delete_model() 
predictor.delete_endpoint()